In [1]:
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate

llm = OllamaLLM(model="llama3.2")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("user", "{question}")
])
chain = prompt | llm 

In [2]:
print(chain.invoke({"question": "Explain Retrieval-Augmented Generation"}))

Retrieval-Augmented Generation (RAG) is an exciting field of research in Natural Language Processing (NLP). I'd be happy to explain it to you.

**What is Retrieval-Augmented Generation?**

Retrieval-Augmented Generation is a class of models that combine the strengths of both generation and retrieval tasks. The core idea is to use pre-trained language models as retrievers, which search for relevant information in a large database or corpus, and then use this retrieved information to inform a subsequent generation task.

**Components of RAG**

A typical RAG system consists of three main components:

1. **Retriever**: This is the pre-trained language model that searches for relevant information in a large database or corpus.
2. **Generator**: This is another pre-trained language model that generates text based on the retrieved information from the retriever.
3. **Evaluator**: This component assesses the quality of the generated text.

**How RAG Works**

Here's an overview of how RAG works

In [3]:
import requests
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

# ----------------------
# Pydantic Schemas
# ----------------------
class PlayerInput(BaseModel):
    name: str = Field(..., description="Name of the player")
    age: int = Field(..., description="Age of the player")
    sport: str = Field(..., description="Sport the player plays")

class UpdatePlayerInput(BaseModel):
    player_id: int = Field(..., description="ID of the player to update")
    name: str = Field(..., description="Updated name of the player")
    age: int = Field(..., description="Updated age of the player")
    sport: str = Field(..., description="Updated sport of the player")

class DeletePlayerInput(BaseModel):
    player_id: int = Field(..., description="ID of the player to delete")

class QueryInput(BaseModel):
    query: str = Field(..., description="A raw SQL SELECT query to run on the database")


# ----------------------
# API Wrappers
# ----------------------
def call_mcp_add(name: str, age: int, sport: str):
    """Call API to add a new player."""
    payload = {"name": name, "age": age, "sport": sport}
    r = requests.post("http://localhost:8000/add_player", json=payload)
    return r.json()

def call_mcp_read():
    """Call API to read all players."""
    r = requests.get("http://localhost:8000/read_players")
    return r.json()

def call_mcp_update(player_id: int, name: str, age: int, sport: str):
    """Call API to update a player by ID."""
    payload = {"name": name, "age": age, "sport": sport}
    r = requests.put(f"http://localhost:8000/update_player/{player_id}", json=payload)
    return r.json()

def call_mcp_delete(player_id: int):
    """Call API to delete a player by ID."""
    r = requests.delete(f"http://localhost:8000/delete_player/{player_id}")
    return r.json()

def call_mcp_query(query: str):
    """Call API to run a raw SQL query (SELECT only)."""
    r = requests.post("http://localhost:8000/run_query", json={"query": query})
    return r.json()


# ----------------------
# LangChain Tools
# ----------------------
mcp_add_tool = StructuredTool.from_function(
    func=call_mcp_add,
    name="add_player",
    description="Add a new player to the SQLite DB.",
    args_schema=PlayerInput
)

mcp_read_tool = StructuredTool.from_function(
    func=call_mcp_read,
    name="read_players",
    description="Read all players from the SQLite DB. No input required."
)

mcp_update_tool = StructuredTool.from_function(
    func=call_mcp_update,
    name="update_player",
    description="Update a player's info by ID.",
    args_schema=UpdatePlayerInput
)

mcp_delete_tool = StructuredTool.from_function(
    func=call_mcp_delete,
    name="delete_player",
    description="Delete a player by ID.",
    args_schema=DeletePlayerInput
)

mcp_query_tool = StructuredTool.from_function(
    func=call_mcp_query,
    name="run_query",
    description="Run a raw SELECT SQL query on the database.",
    args_schema=QueryInput
)


In [14]:
system_prompt = """
You are a database agent specialized in managing a SQLite database of players.

Rules:
- ONLY use the provided tools to interact with the database.
- When calling a tool, output a valid JSON object.
- Do NOT include field descriptions or types, only actual values.
- Do NOT add extra explanations or commentary.
- Always select the tool that exactly matches the user request.
- Fill all required tool_input fields with real values.

Use-case Examples:

1️⃣ Add a new player:
User: "Add a new player named Alice, 25 years old, who plays Football."
Agent:
{
  "tool": "add_player",
  "tool_input": {
    "name": "Alice",
    "age": 25,
    "sport": "Football"
  }
}

2️⃣ Read all players:
User: "Show me all players in the database."
Agent:
{
  "tool": "read_players",
  "tool_input": {}
}

3️⃣ Update a player's info:
User: "Update player with ID 1, change name to Alicia, age to 26, sport to Basketball."
Agent:
{
  "tool": "update_player",
  "tool_input": {
    "player_id": 1,
    "name": "Alicia",
    "age": 26,
    "sport": "Basketball"
  }
}

4️⃣ Delete a player:
User: "Delete the player with ID 2."
Agent:
{
  "tool": "delete_player",
  "tool_input": {
    "player_id": 2
  }
}

5️⃣ Run a custom SQL query:
User: "Get the names and sports of players older than 20."
Agent:
{
  "tool": "run_query",
  "tool_input": {
    "query": "SELECT name, sport FROM player WHERE age > 20"
  }
}

6️⃣ Combined scenario:
User: "Add a player named Bob, 30, Tennis, then show me all players."
Agent should first call "add_player" with Bob's info, then "read_players" in sequence, each with correct JSON.

Always follow these use-case patterns. Fill tool_input fields correctly with actual values, and never include schema or explanations.
"""


In [15]:
from langchain.agents import initialize_agent, AgentType


llm = OllamaLLM(model="mistral")

agent = initialize_agent(
    tools=[
        mcp_add_tool,
        mcp_read_tool,
        mcp_update_tool,
        mcp_delete_tool,
        mcp_query_tool
    ],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"system_message": system_prompt},
    verbose=True,
    handle_parsing_errors=True
)

In [16]:
# Add new player
agent.invoke("Show all the players in database")



> Entering new AgentExecutor chain...


 Thought: I should use the `read_players` tool to display all the players in the SQLite DB.
Action:
```
{
  "action": "read_players",
  "action_input": {}
}
```

Observation: {'data': [[1, 'Alice', 25, 'Football'], [2, 'Bob', 30, 'Tennis'], [3, 'Charlie', 22, 'Basketball'], [4, 'Alice (Duplicate)', 25, 'Football']]}
Thought: The SQLite database contains the following players: Alice (Football, age 25), Bob (Tennis, age 30), Charlie (Basketball, age 22), and Alice (a duplicate of the first player).

> Finished chain.


{'input': 'Show all the players in database',
 'output': ' The SQLite database contains the following players: Alice (Football, age 25), Bob (Tennis, age 30), Charlie (Basketball, age 22), and Alice (a duplicate of the first player).'}

In [19]:
agent.invoke("Show all the players in the database")



> Entering new AgentExecutor chain...


 Thought: The human wants to view all players in the database, so I will use the `read_players` tool.

Action:
```
{
  "action": "read_players",
  "action_input": {}
}
```

Observation: {'data': [[1, 'Alice', 25, 'Football'], [2, 'Bob', 30, 'Tennis'], [3, 'Charlie', 22, 'Basketball']]}
Thought: I found the players in the database. Here they are:

1. Alice (age 25, plays Football)
2. Bob (age 30, plays Tennis)
3. Charlie (age 22, plays Basketball)

If you need more information about a specific player, let me know!

Thought: My job is done here, so I will provide the final answer.

Action:
```
{
  "action": "Final Answer",
  "action_input": "I found the players in the database. Here they are:\n1. Alice (age 25, plays Football)\n2. Bob (age 30, plays Tennis)\n3. Charlie (age 22, plays Basketball)"
}
```

> Finished chain.


{'input': 'Show all the players in the database',
 'output': 'I found the players in the database. Here they are:\n1. Alice (age 25, plays Football)\n2. Bob (age 30, plays Tennis)\n3. Charlie (age 22, plays Basketball)'}